In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida

from aiida.engine import ProcessState


import ipywidgets as ipw
import traitlets
import warnings

from IPython.display import clear_output

from aiidalab_widgets_base import OptimadeQueryWidget
from aiidalab_widgets_base import StructureBrowserWidget
from aiidalab_widgets_base import StructureManagerWidget
from aiidalab_widgets_base import StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import ComputationalResources
from aiidalab_widgets_base import SubmitButtonWidget
from aiidalab_widgets_base import ProcessNodesTreeWidget
from aiidalab_widgets_base import ProcessMonitor
from aiidalab_widgets_base import viewer
from aiidalab_widgets_base import register_viewer_widget

from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler



import aiida_nanotech_empa.utils.gaussian_wcs_postprocess as pp
StructureData = DataFactory("structure")
GaussianSpinWorkChain = WorkflowFactory('nanotech_empa.gaussian.spin')

from aiidalab_widgets_base import WizardAppWidget
from aiidalab_widgets_base import WizardAppWidgetStep

In [ ]:
class NodeViewWidget(ipw.VBox):

    node = traitlets.Instance(Node, allow_none=True)

    def __init__(self, **kwargs):
        self._output = ipw.Output()
        super().__init__(children=[self._output], **kwargs)

    @traitlets.observe("node")
    def _observe_node(self, change):
        if change["new"] != change["old"]:
            with self._output:
                clear_output()
                if change["new"]:
                    display(viewer(change["new"]))

In [ ]:
from threading import Event, Lock, Thread
from dataclasses import dataclass
from aiida.cmdline.utils.query.calculation import CalculationQueryBuilder

class WorkChainSelector(ipw.HBox):

    # The PK of a 'aiida.workflows:quantumespresso.pw.bands' WorkChainNode.
    value = traitlets.Int(allow_none=True)

    # When this trait is set to a positive value, the work chains are automatically
    # refreshed every `auto_refresh_interval` seconds.
    auto_refresh_interval = traitlets.Int()  # seconds

    # Indicate whether the widget is currently updating the work chain options.
    busy = traitlets.Bool(read_only=True)

    # Note: We use this class as a singleton to reset the work chains selector
    # widget to its default stage (no work chain selected), because we cannot
    # use `None` as setting the widget's value to None will lead to "no selection".
    _NO_PROCESS = object()

    FMT_WORKCHAIN = "{wc.pk:6}{wc.ctime:>10}\t{wc.state:<16}\t{wc.formula}"

    def __init__(self, **kwargs):
        self.work_chains_prompt = ipw.HTML("<b>Select workflow or start new:</b>&nbsp;")
        self.work_chains_selector = ipw.Dropdown(
            options=[("New workflow...", self._NO_PROCESS)],
            layout=ipw.Layout(min_width="300px", flex="1 1 auto"),
        )
        ipw.dlink(
            (self.work_chains_selector, "value"),
            (self, "value"),
            transform=lambda pk: None if pk is self._NO_PROCESS else pk,
        )

        self.refresh_work_chains_button = ipw.Button(description="Refresh")
        self.refresh_work_chains_button.on_click(self.refresh_work_chains)

        self._refresh_lock = Lock()
        self._refresh_thread = None
        self._stop_refresh_thread = Event()
        self._update_auto_refresh_thread_state()

        super().__init__(
            children=[
                self.work_chains_prompt,
                self.work_chains_selector,
                self.refresh_work_chains_button,
            ],
            **kwargs,
        )

    @dataclass
    class WorkChainData:
        pk: int
        ctime: str
        state: str
        formula: str

    @classmethod
    def find_work_chains(cls):
        builder = CalculationQueryBuilder()
        filters = builder.get_filters(
            process_label="GaussianSpinWorkChain",
        )
        query_set = builder.get_query_set(
            filters=filters,
            order_by={"ctime": "desc"},
        )
        projected = builder.get_projected(
            query_set, projections=["pk", "ctime", "state"]
        )

        for process in projected[1:]:
            pk = process[0]
            formula = load_node(pk).inputs.structure.get_formula()
            yield cls.WorkChainData(formula=formula, *process)

    @traitlets.default("busy")
    def _default_busy(self):
        return True

    @traitlets.observe("busy")
    def _observe_busy(self, change):
        for child in self.children:
            child.disabled = change["new"]

    def refresh_work_chains(self, _=None):
        with self._refresh_lock:
            try:
                self.set_trait("busy", True)  # disables the widget

                with self.hold_trait_notifications():
                    # We need to restore the original value, because it may be reset due to this issue:
                    # https://github.com/jupyter-widgets/ipywidgets/issues/2230
                    original_value = self.work_chains_selector.value

                    self.work_chains_selector.options = [
                        ("New calculation...", self._NO_PROCESS)
                    ] + [
                        (self.FMT_WORKCHAIN.format(wc=wc), wc.pk)
                        for wc in self.find_work_chains()
                    ]

                    self.work_chains_selector.value = original_value
            finally:
                self.set_trait("busy", False)  # reenable the widget

    def _auto_refresh_loop(self):
        while True:
            self.refresh_work_chains()
            if self._stop_refresh_thread.wait(timeout=self.auto_refresh_interval):
                break

    def _update_auto_refresh_thread_state(self):
        if self.auto_refresh_interval > 0 and self._refresh_thread is None:
            # start thread
            self._stop_refresh_thread.clear()
            self._refresh_thread = Thread(target=self._auto_refresh_loop)
            self._refresh_thread.start()

        elif self.auto_refresh_interval <= 0 and self._refresh_thread is not None:
            # stop thread
            self._stop_refresh_thread.set()
            self._refresh_thread.join(timeout=30)
            self._refresh_thread = None

    @traitlets.default("auto_refresh_interval")
    def _default_auto_refresh_interval(self):
        return 10  # seconds

    @traitlets.observe("auto_refresh_interval")
    def _observe_auto_refresh_interval(self, change):
        if change["new"] != change["old"]:
            self._update_auto_refresh_thread_state()

    @traitlets.observe("value")
    def _observe_value(self, change):
        if change["old"] == change["new"]:
            return

        new = self._NO_PROCESS if change["new"] is None else change["new"]

        if new not in {pk for _, pk in self.work_chains_selector.options}:
            self.refresh_work_chains()

        self.work_chains_selector.value = new


In [ ]:

class StructureSelectionStep(ipw.VBox, WizardAppWidgetStep):
    """Integrated widget for the selection of structures from different sources."""

    confirmed_structure = traitlets.Instance(StructureData, allow_none=True)

    def __init__(self, description=None, **kwargs):
        self.manager = StructureManagerWidget(
            importers=[
                StructureUploadWidget(title="From computer"),
                OptimadeQueryWidget(embedded=True),
                StructureBrowserWidget(title="AiiDA database"),
                SmilesWidget(title="SMILES")
            ],
            node_class='StructureData',
        )
        self.manager.observe(self._update_state, ["structure_node"])

        if description is None:
            description = ipw.HTML(
                """
                <p>Select a structure from one of the following sources and then click
                "Confirm" to go to the next step.
                """
            )
        self.description = description


        self.confirm_button = ipw.Button(
            description="Confirm",
            tooltip="Confirm the currently selected structure and go to the next step.",
            button_style="success",
            icon="check-circle",
            disabled=True,
            layout=ipw.Layout(width="auto"),
        )
        self.confirm_button.on_click(self.confirm)

        super().__init__(
            children=[
                self.description,
                self.manager,
                self.confirm_button,
            ],
            **kwargs
        )

    @traitlets.default("state")
    def _default_state(self):
        return self.State.READY

    def _update_state(self, _=None):
        if self.manager.structure_node is None:
            if self.confirmed_structure is None:
                self.state = self.State.READY
            else:
                self.state = self.State.SUCCESS
        else:
            if self.confirmed_structure is None:
                self.state = self.State.CONFIGURED
            else:
                self.state = self.State.SUCCESS

    @traitlets.observe("confirmed_structure")
    def _observe_confirmed_structure(self, _):
        with self.hold_trait_notifications():
            self._update_state()

    @traitlets.observe("state")
    def _observe_state(self, change):
        with self.hold_trait_notifications():
            state = change["new"]
            self.confirm_button.disabled = state != self.State.CONFIGURED
            self.manager.disabled = state is self.State.SUCCESS

    def confirm(self, _=None):
        self.manager.store_structure()
        self.confirmed_structure = self.manager.structure_node

    def can_reset(self):
        return self.confirmed_structure is not None

    def reset(self):  # unconfirm
        self.confirmed_structure = None
        self.manager.structure = None
        
select_structure_step = StructureSelectionStep(auto_advance=True)

In [ ]:
class ConfigureGaussianCalculationStep(ipw.VBox, WizardAppWidgetStep):
    """Widget to prepare gaussian inputs."""


    inputs = traitlets.Dict()
    input_structure = traitlets.Instance(StructureData, allow_none=True)

    def __init__(self, **kwargs):

        self.dft_functional = ipw.Dropdown(description="DFT functional:", value='B3LYP', options=['B3LYP', 'PBE', 'PBE0'], style={'description_width':'initial'})
        self.empirical_dispersion = ipw.Dropdown(description="Empirical dispersion:", options=['GD3', 'GD3BJ'], style={'description_width':'initial'})
        basis_sets = ['STO-3G', "3-21G", "6-21G", "6-31G", "6-311G", "6-311+G"]
        self.basis_set_opt = ipw.Dropdown(description="Basis set for optimization:", options=basis_sets, style={'description_width':'initial'})
        self.basis_set_scf = ipw.Dropdown(description="Basis set for SCF:", options=basis_sets, style={'description_width':'initial'})
        self.multiplicity_list = ipw.Text(description="Multiplicity list:", value="1", style={'description_width':'initial'})

        self.observe(self._update_state, ["inputs", "input_structure"])
        
        self.confirm_button = ipw.Button(
            description="Confirm",
            tooltip="Confirm the currently selected structure and go to the next step.",
            button_style="success",
            icon="check-circle",
            disabled=False,
            layout=ipw.Layout(width="auto"),
        )
        self.confirm_button.on_click(self.confirm)

        super().__init__([self.dft_functional, self.empirical_dispersion, self.basis_set_opt, self.basis_set_scf, self.multiplicity_list, self.confirm_button], **kwargs)
        
    def reset(self):
        self.inputs = {}

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.input_structure:  # the order can be submitted
            self.state = self.State.READY
        else:
            self.state = self.State.INIT

    def confirm(self, _=None):
        self.inputs = dict(
            functional = Str(self.dft_functional.value),
            empirical_dispersion = Str(self.empirical_dispersion.value),
            basis_set_opt = Str(self.basis_set_opt.value),
            basis_set_scf = Str(self.basis_set_scf.value),
            multiplicity_list = List(list=list(map(int, self.multiplicity_list.value.split()))),
            structure = self.input_structure
        )
        self.state = self.State.SUCCESS
    
    @traitlets.default("state")
    def _default_state(self):
        return self.State.INIT

configure_calculation_step = ConfigureGaussianCalculationStep(auto_advance=True)

_ = ipw.dlink((select_structure_step, 'confirmed_structure'), (configure_calculation_step, 'input_structure'))

In [ ]:
class SubmitGaussianCalculationStep(ipw.VBox, WizardAppWidgetStep):
    """Integrated widget for the selection of structures from different sources."""


    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations, they do not need to be of the same type.
    inputs = traitlets.Dict()
    process = traitlets.Instance(ProcessNode, allow_none=True)

    def __init__(self, **kwargs):
        self.configuration_label = ipw.HTML("Specify computational resources.")
        
        # Codes.
        self.gaussian_code_dropdown = ComputationalResources(description="Gaussian", input_plugin='gaussian')
        self.gaussian_code_dropdown.observe(self._update_state, ["selected_code"])
        self.cubegen_code_dropdown = ComputationalResources(description="Cubegen", input_plugin='gaussian.cubegen')
        self.cubegen_code_dropdown.observe(self._update_state, ["selected_code"])
        self.formchk_code_dropdown = ComputationalResources(description="Formchk", input_plugin='gaussian.formchk')
        self.formchk_code_dropdown.observe(self._update_state, ["selected_code"])

        
        # Resournces.
        self.n_mpi_tasks_widget = ipw.IntText(description="# MPI tasks", value=1, min=1, style={"description_width": "100px"})
        self.memory_widget = ipw.IntText(description="Memory (Mb)", value=300, step=100, min=0, style={"description_width": "100px"})
        self.run_time_widget = ipw.IntText(description="Runtime (mins)", value=120, min=0, style={"description_width": "100px"})

        # We update the step's state whenever there is a change to the configuration or the order status.
        self.observe(self._update_state, ["inputs"])
        
        self.btn_submit_mol_opt = SubmitButtonWidget(GaussianSpinWorkChain, input_dictionary_function=self.prepare_spin_calc)
        self.btn_submit_mol_opt.btn_submit.disabled = True
        traitlets.dlink((self.btn_submit_mol_opt, 'process'), (self, 'process'))

        super().__init__([ipw.HBox([ipw.VBox([self.gaussian_code_dropdown,
                          self.cubegen_code_dropdown,
                          self.formchk_code_dropdown]), ipw.VBox([self.n_mpi_tasks_widget, self.memory_widget, self.run_time_widget])]), self.btn_submit_mol_opt], **kwargs)
        
    def reset(self):
        self.inputs ={}
        self.state = self.State.INIT

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.inputs:

            # All codes are provided.
            if self.gaussian_code_dropdown.selected_code and self.cubegen_code_dropdown.selected_code and self.formchk_code_dropdown.selected_code:
                self.state = self.State.CONFIGURED
                self.btn_submit_mol_opt.btn_submit.disabled = False
            else:
                self.state = self.State.READY
                self.btn_submit_mol_opt.btn_submit.disabled = True

        else:
            self.state = self.State.INIT
            self.btn_submit_mol_opt.btn_submit.disabled = True
    
    def prepare_spin_calc(self):
        builder = GaussianSpinWorkChain.get_builder()
        

        # Input nodes.
        for key, value in self.inputs.items():
            builder[key] = value

        # Codes.
        builder.gaussian_code = self.gaussian_code_dropdown.selected_code
        builder.formchk_code = self.formchk_code_dropdown.selected_code
        builder.cubegen_code = self.cubegen_code_dropdown.selected_code
        
        # Resources.
        builder.options =  Dict(dict={
            "resources": {
                "num_machines": 1,
                "tot_num_mpiprocs": self.n_mpi_tasks_widget.value,
                },
            "max_memory_kb": int(1.25 * self.memory_widget.value) * 1024,
            "max_wallclock_seconds": 60 * self.run_time_widget.value,
            })
        
        
        self.state = self.State.SUCCESS

        return builder

submit_calculation_step = SubmitGaussianCalculationStep(auto_advance=True)
_ = ipw.dlink((configure_calculation_step, 'inputs'), (submit_calculation_step, "inputs"))

In [ ]:
class ViewGaussianWorkChainStatusAndResultsStep(ipw.VBox, WizardAppWidgetStep):

    process = traitlets.Instance(ProcessNode, allow_none=True)

    def __init__(self, **kwargs):
        self.process_tree = ProcessNodesTreeWidget()
        ipw.dlink((self, "process"), (self.process_tree, "process"))

        self.node_view = NodeViewWidget(layout={"width": "auto", "height": "auto"})
        ipw.dlink(
            (self.process_tree, "selected_nodes"),
            (self.node_view, "node"),
            transform=lambda nodes: nodes[0] if nodes else None,
        )
        self.process_status = ipw.VBox(children=[self.process_tree, self.node_view])

        # Setup process monitor
        self.process_monitor = ProcessMonitor(
            timeout=0.2,
            callbacks=[
                self.process_tree.update,
                self._update_state,
            ],
        )
        ipw.dlink((self, "process"), (self.process_monitor, "process"))

        super().__init__([self.process_status], **kwargs)

    def can_reset(self):
        "Do not allow reset while process is running."
        return self.state is not self.State.ACTIVE

    def reset(self):
        self.process = None

    def _update_state(self):
        if self.process is None:
            self.state = self.State.INIT
        else:
            process_state = self.process.process_state
            if process_state in (
                ProcessState.CREATED,
                ProcessState.RUNNING,
                ProcessState.WAITING,
            ):
                self.state = self.State.ACTIVE
            elif process_state in (ProcessState.EXCEPTED, ProcessState.KILLED):
                self.state = self.State.FAIL
            elif process_state is ProcessState.FINISHED:
                self.state = self.State.SUCCESS

    @traitlets.observe("process")
    def _observe_process(self, change):
        self._update_state()

results = ViewGaussianWorkChainStatusAndResultsStep(auto_advance=True)
_ = ipw.dlink((submit_calculation_step, 'process'), (results, 'process'))

In [ ]:
# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)

        # Structure:
        with select_structure_step.manager.hold_sync():
            with select_structure_step.hold_sync():
                select_structure_step.confirmed_structure = process.inputs.structure
                select_structure_step.manager.input_structure = process.inputs.structure
        # Config.
        configure_calculation_step.dft_functional.value = process.inputs.functional
        configure_calculation_step.empirical_dispersion.value = process.inputs.empirical_dispersion
        configure_calculation_step.basis_set_opt.value = process.inputs.basis_set_opt
        configure_calculation_step.basis_set_scf.value = process.inputs.basis_set_scf
        configure_calculation_step.multiplicity_list.value = " ".join(map(str, process.inputs.multiplicity_list))
        configure_calculation_step.state = WizardAppWidgetStep.State.SUCCESS
        configure_calculation_step.confirm_button.disabled = True

        # Codes & resources
        submit_calculation_step.gaussian_code_dropdown.selected_code = process.inputs.gaussian_code
        submit_calculation_step.formchk_code_dropdown.selected_code = process.inputs.formchk_code
        submit_calculation_step.cubegen_code_dropdown.selected_code = process.inputs.cubegen_code

        if "options" in process.inputs:
            submit_calculation_step.n_mpi_tasks_widget.value = process.inputs.options["resources"]["tot_num_mpiprocs"]
            submit_calculation_step.memory_widget.value = process.inputs.options["max_memory_kb"] / 1024 / 1.25
            submit_calculation_step.run_time_widget.value = process.inputs.options["max_wallclock_seconds"] / 60
        submit_calculation_step.state = WizardAppWidgetStep.State.SUCCESS

        # Running process.
        submit_calculation_step.process = process
        
work_chain_selector.observe(_observe_process_selection, 'value')    
_ = ipw.dlink((submit_calculation_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

In [ ]:
app = WizardAppWidget(
    steps=[
        ("Select structure", select_structure_step),
        ("Cofigure calculation", configure_calculation_step),
        ("Submit", submit_calculation_step),
        ("Results", results)
    ]
)

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/nanotech-empa/aiidalab-empa-molecules/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(app_with_work_chain_selector)
    
display(output)

In [ ]:
class SummaryView(ipw.VBox):
    def __init__(self, wc_node, **kwargs):

        self.wc_node = wc_node

        def _fmt_yes_no(truthy):
            return "Yes" if truthy else "No"

        report = generate_report_dict(self.wc_node)

        env = Environment()
        env.filters.update(
            {
                "fmt_yes_no": _fmt_yes_no,
            }
        )
        template = resources.read_text(static, "workflow_summary.jinja")
        style = resources.read_text(static, "style.css")
        self.summary_view = ipw.HTML(
            env.from_string(template).render(style=style, **report)
        )
        super().__init__(
            children=[self.summary_view],
            **kwargs,
        )


@register_viewer_widget("process.workflow.workchain.WorkChainNode.")
class WorkChainViewer(ipw.VBox):

    def __init__(self, node, **kwargs):
        if node.process_label != "GaussianSpinWorkChain":
            raise KeyError(str(node.node_type))

        self.node = node

        self.title = ipw.HTML(
            f"""
            <hr style="height:2px;background-color:#2097F3;">
            <h4>Gaussian Spin Work Chain (pk: {self.node.pk}) &mdash;
                {self.node.inputs.structure.get_formula()}
            </h4>
            """
        )
        
        self._output = ipw.Output()

        with self._output:
            clear_output()
            if node.process_state in (
                ProcessState.CREATED,
                ProcessState.RUNNING,
                ProcessState.WAITING,
            ):
                display(ipw.HTML("Simulation is still running, no results shown (yet)."))
            elif node.process_state in (ProcessState.EXCEPTED, ProcessState.KILLED):
                display(ipw.HTML("Simulation couldn't be completed, sorry."))
            elif node.process_state is ProcessState.FINISHED:
                if node.exit_status == 0:
                    pp.make_report(node, nb=True)
                else:
                    display(ipw.HTML("Simulation is completed, but has a non-zero exit status."))

        super().__init__(
            children=[self.title, self._output],
            **kwargs,
        )

